In [1]:
from transformers import AutoModel,AutoTokenizer,AutoModelForMaskedLM
import os
import sys
import h5py
from tqdm import tqdm
import torch
import numpy as np
sys.path.append('../../data_generation/')
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
file = h5py.File('../../data/rna_stable/insert_dataset.h5','r')

In [2]:
model = AutoModel.from_pretrained('zhihan1996/DNA_bert_6',output_hidden_states=True).to('cuda')
tokenizer = AutoTokenizer.from_pretrained('zhihan1996/DNA_bert_6')

Some weights of the model checkpoint at zhihan1996/DNA_bert_6 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
dnabert_output = h5py.File('../../data/rna_stable/dnabert_embed.h5','w')
batch_size = 32
for dataset in ['test','train','valid']:
    key = 'X_'+dataset
    onehot = file[key]
    string_seq = utils.onehot_to_seq(onehot)
    split_seq = []
    for seq in string_seq:
        split_seq.append(' '.join([seq[i:i+6] for i in range(0, len(seq)-6+1, 1)]))
    
    token_seq = tokenizer.batch_encode_plus(split_seq,max_length=507)
    output_cache = []
    for seq_i in tqdm(range(0,len(token_seq['input_ids']),batch_size)):
        seq_batch = torch.tensor(token_seq['input_ids'][seq_i:seq_i+batch_size]).to('cuda')
        output_seq = model(seq_batch)[-1][-1].cpu().detach().numpy()
        output_cache.extend(output_seq)
    dnabert_output.create_dataset(name=key,data = np.array(output_cache))
    dnabert_output.create_dataset(name='Y_'+dataset,data = file['Y_'+dataset][:])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
100%|██████████| 36/36 [00:02<00:00, 13.39it/s]


In [4]:
dnabert_output.keys()

<KeysViewHDF5 ['X_test', 'X_train', 'X_valid', 'Y_test', 'Y_train', 'Y_valid']>

In [5]:
dnabert_output.close()